In [2]:
import numpy as np
import pandas as pd
import nltk
nltk.download('stopwords')
import os
import gensim


[nltk_data] Downloading package stopwords to /Users/gea/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
df_train = pd.read_csv('data_worthcheck/train.csv')
df_train.head()

,Unnamed: 0,text_a,label
0,0,betewe buka twitter cuman ngetweet liat home b...,no
1,1,mas piyuuu mugo2 corona tuh mulut tersumpal ma...,no
2,2,e100ss gini buka informasi sejelas nya identit...,yes
3,3,neng solo wes ono terduga corona cobo neng ati...,no
4,4,midiahn nii akun gak takut takut nya isu coron...,no


In [77]:
embeddings = gensim.models.Word2Vec(df_train['text_a'], vector_size=100, window=3, min_count=2)

In [78]:
docs_vectors_train = pd.DataFrame()
stopwords = nltk.corpus.stopwords.words('indonesian')
for doc in df_train['text_a'].str.lower().str.replace('[^a-z ]', ''):
    temp = pd.DataFrame()
    for word in doc.split(' '):
        if word not in stopwords:
            try:
                word_vec = embeddings.wv[word]
                temp = temp.append(pd.Series(word_vec), ignore_index = True)
            except:
                pass
    doc_vector_train = temp.mean()
    docs_vectors_train = docs_vectors_train.append(doc_vector_train, ignore_index = True)
docs_vectors_train.shape

(21601, 100)

In [79]:
pd.isnull(docs_vectors_train).sum().sum()

1620100

In [80]:
docs_vectors_train['label'] = df_train['label']
docs_vectors_train = docs_vectors_train.dropna()

In [81]:
df_test = pd.read_csv('data_worthcheck/test.csv')
df_test.head()

,text_a,label
0,jek dajal ga depok bang,no
1,detikcom untung depok masuk wilayah nya ridwan...,no
2,df dom jakarta depok yg gunain vc cabang nya c...,no
3,your2rl depok jkt,no
4,doakan indonesia selamat virus corona pkb depo...,yes


In [82]:
docs_vectors_test = pd.DataFrame()
stopwords = nltk.corpus.stopwords.words('indonesian')
for doc in df_test['text_a'].str.lower().str.replace('[^a-z ]', ''):
    temp = pd.DataFrame()
    for word in doc.split(' '):
        if word not in stopwords:
            try:
                word_vec = embeddings.wv[word]
                temp = temp.append(pd.Series(word_vec), ignore_index = True)
            except:
                pass
    doc_vector_test = temp.mean()
    docs_vectors_test = docs_vectors_test.append(doc_vector_test, ignore_index = True)
docs_vectors_test.shape

(2800, 100)

In [83]:
pd.isnull(docs_vectors_test).sum().sum()

226500

In [84]:
docs_vectors_test['label'] = df_test['label']
docs_vectors_test = docs_vectors_test.dropna()

In [85]:
from sklearn.ensemble import AdaBoostClassifier

train_x = docs_vectors_train.drop('label', axis=1)
train_y = docs_vectors_train['label']
test_x = docs_vectors_test.drop('label', axis=1)
test_y = docs_vectors_test['label']

In [86]:
from sklearn.metrics import accuracy_score

model = AdaBoostClassifier(n_estimators=800, random_state = 1)
model.fit(train_x, train_y)
test_pred = model.predict(test_x)
accuracy_score(test_y, test_pred)


0.6074766355140186